<a href="https://colab.research.google.com/github/tanu-ns/NLP-Project/blob/main/dataframe_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import math
import nltk
nltk.download('indian')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import tnt
from nltk.corpus import indian
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm

[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Package indian is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Minor Project/Hin_train.csv')
df.head()

,id,Link,Heading,Summary,Article
0,a3d72cd62e7e1b5529dd254a85fa2b5a,https://www.indiatv.in/india/national/india-su...,"भारत की बढ़ी ताकत, ‘अग्नि प्राइम’ मिसाइल का हु...",भारत ने शनिवार को ओडिशा के बालासोर तट पर ‘अग्न...,भुवनेश्वर : भारत ने शनिवार को ओडिशा के बालासोर...
1,f8904fd63c1d257dc5ee9b529df4a097,https://www.indiatv.in/india/national-covid-19...,Covid-19 Vaccination: देश में कोविड टीके की 40...,"केंद्रीय स्वास्थ्य मंत्रालय के मुताबिक, आज शाम...",नई दिल्ली। देश में कोरोना की संभावित तीसरी लहर...
2,4e89ee2300613ae0c0e8db4ed0e2c7aa,https://www.indiatv.in/india/uttar-pradesh/bjp...,यूपी में ब्राह्मण वोटरों को साधने की तैयारी कर...,"बीजेपी के केंद्र सरकार में ब्राह्मण मंत्रियों,...",बीजेपी यूपी में जनवरी के दूसरे हफ्ते से ब्राह्...
3,9a382557dc6afda0dfaf04210d16224f,https://www.indiatv.in/india/uttar-pradesh-si-...,बागपत में बिना इजाजत दाढ़ी रखने पर दरोगा सस्पे...,उत्तर प्रदेश के बागपत में बिना अनुमति के बड़ी ...,बागपत: उत्तर प्रदेश के बागपत में बिना अनुमति क...
4,928dcc9264915737630ab0e32f4a9b28,https://www.indiatv.in/india/politics-farm-law...,कृषि क़ानूनों की वापसी की घोषणा पर मैं अवाक रह...,उमा भारती ने कहा कि अगर तीन कृषि कानूनों की मह...,भोपाल: मध्य प्रदेश की पूर्व मुख्यमंत्री और बीज...


In [ ]:
df.drop(
    ['id','Link'],
    axis = 1,
    inplace = True
)

In [ ]:
df.head()

,Heading,Summary,Article
0,"भारत की बढ़ी ताकत, ‘अग्नि प्राइम’ मिसाइल का हु...",भारत ने शनिवार को ओडिशा के बालासोर तट पर ‘अग्न...,भुवनेश्वर : भारत ने शनिवार को ओडिशा के बालासोर...
1,Covid-19 Vaccination: देश में कोविड टीके की 40...,"केंद्रीय स्वास्थ्य मंत्रालय के मुताबिक, आज शाम...",नई दिल्ली। देश में कोरोना की संभावित तीसरी लहर...
2,यूपी में ब्राह्मण वोटरों को साधने की तैयारी कर...,"बीजेपी के केंद्र सरकार में ब्राह्मण मंत्रियों,...",बीजेपी यूपी में जनवरी के दूसरे हफ्ते से ब्राह्...
3,बागपत में बिना इजाजत दाढ़ी रखने पर दरोगा सस्पे...,उत्तर प्रदेश के बागपत में बिना अनुमति के बड़ी ...,बागपत: उत्तर प्रदेश के बागपत में बिना अनुमति क...
4,कृषि क़ानूनों की वापसी की घोषणा पर मैं अवाक रह...,उमा भारती ने कहा कि अगर तीन कृषि कानूनों की मह...,भोपाल: मध्य प्रदेश की पूर्व मुख्यमंत्री और बीज...


In [ ]:
df['Article'][0]

'भुवनेश्वर : भारत ने शनिवार को ओडिशा के बालासोर तट पर ‘अग्नि प्राइम’ मिसाइल का सफल परीक्षण किया। सरकारी अधिकारियों ने इसकी जानकारी दी। अग्नि प्राइम मिसाइल अग्नि सीरीज की नए जनरेशन वाली एडवांस मिसाइल है। इसकी मारक क्षमता 1000 से 2000 किलोमीटर के बीच है। मिसाइल को रक्षा अनुसंधान और विकास संगठन (DRDO) द्वारा डिजाइन और विकसित किया गया है। अग्नि प्राइम अल्\u200dट्रा मॉर्डन टेक्निक से लैस होने की वजह से बहुत हल्के वजन की मिसाइल है।आपको बता दें कि इसी सप्ताह भारत ‘सुपरसोनिक मिसाइल असिस्टेड टॉरपीडो सिस्टम’ का भी सफलतापूर्वक प्रक्षेपण किया था। इस मिसाइल का प्रक्षेपण ओडिशा तट से दूर अब्दुल कलाम द्वीप से किया गया था।'

In [ ]:
df['Summary'][0]

'भारत ने शनिवार को ओडिशा के बालासोर तट पर ‘अग्नि प्राइम’ मिसाइल का सफल परीक्षण किया। सरकारी अधिकारियों ने इसकी जानकारी दी। अग्नि प्राइम मिसाइल अग्नि सीरीज की नए जनरेशन वाली एडवांस मिसाइल है।'

In [ ]:
from evaluate import load
bertscore = load("bertscore")

import nltk
nltk.download("punkt")

from bert_score import score

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np

import string

from evaluate import load
bertscore = load("bertscore")

global_R = []

for d in range(1000):
  article_sent = df['Article'][d]
  article_sent = article_sent.split('।')
  if(len(article_sent) > 1):
    del article_sent[-1]

  summary_sent = df[ 'Summary'][d]
  summary_sent = summary_sent.split('।')
  if(len(summary_sent) > 1):
    del summary_sent[-1]


  for i in range(len(article_sent)):
    local_R = []
    for j in range(len(summary_sent)):
      results = bertscore.compute(predictions = [str(article_sent[i])], references = [str(summary_sent[j])], model_type = "distilbert-base-uncased")
      local_R.append(results['recall'])

    global_R.append(round(np.max(local_R),3))

In [ ]:
len(global_R)

In [ ]:
X = pd.read_csv('/content/drive/MyDrive/Minor Project/features.csv')

In [ ]:
size = len(global_R)
X = X[:size]

In [ ]:
X['Label'] = global_R

In [ ]:
X.head()

,Unnamed: 0,Article,Sentence,Sentence Position,Sentence Length,Numeric Data,Thematic Words,Entropy,POS tags,TF-ISF,Aggregate Score,Busy Path,Title,Label
0,0,A1,S1,0.75,0.67,0.0,1.00,1.00,0.87,0.91,0.96,0.33,1.00,0.974
1,1,A1,S2,0.50,0.67,0.0,0.23,0.34,0.28,0.35,0.79,1.00,0.00,1.000
2,2,A1,S3,0.25,1.00,0.0,0.69,0.59,0.62,0.64,0.97,0.17,0.50,1.000
3,3,A1,S4,0.00,0.89,1.0,0.46,0.55,0.33,0.53,0.93,0.33,1.00,0.867
4,4,A1,S5,0.25,0.83,0.0,0.77,0.81,0.77,0.92,0.96,0.33,0.17,0.902


In [ ]:
X.to_csv('new_labeled_dataset.csv')

In [ ]:
len(X)

17409

In [ ]:
X.drop(['Article','Sentence','Unnamed: 0'],axis=1,inplace=True)
X.head()

,Sentence Position,Sentence Length,Numeric Data,Thematic Words,Entropy,POS tags,TF-ISF,Aggregate Score,Busy Path,Title,Label
0,0.75,0.67,0.0,1.00,1.00,0.87,0.91,0.96,0.33,1.00,0.974
1,0.50,0.67,0.0,0.23,0.34,0.28,0.35,0.79,1.00,0.00,1.000
2,0.25,1.00,0.0,0.69,0.59,0.62,0.64,0.97,0.17,0.50,1.000
3,0.00,0.89,1.0,0.46,0.55,0.33,0.53,0.93,0.33,1.00,0.867
4,0.25,0.83,0.0,0.77,0.81,0.77,0.92,0.96,0.33,0.17,0.902
